In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip,BiSkip


In [2]:
import pandas as pd
from Vocabulary import Vocabulary, preprocess

In [3]:
def make_vocab(tokens):
    voc = Vocabulary(['<PAD>','<UNK>'])
    voc.add_tokens(tokens)
    print('vocab len is {}'.format(len(voc.w2idx)))
    return voc

In [4]:
def load_data(file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv'):
    df= pd.read_csv(file)
    df = df.drop('InputStoryid',axis=1)
    targets = df['AnswerRightEnding']
    df = df.drop('AnswerRightEnding',axis=1)
    df = df.drop('InputSentence1',axis=1)
    df = df.drop('InputSentence2',axis=1)
    df = df.drop('InputSentence3',axis=1)
    
    voc_str= ''
    for index, row in df.iterrows():
        voc_str+=' '.join(list(row)) + ' '
        
    df['AnswerRightEnding'] = targets
    return df,make_vocab(preprocess(voc_str))

In [5]:
from sklearn.model_selection import train_test_split
df, voc = load_data()
train, val = train_test_split(df, test_size=0.1,shuffle=False)

vocab len is 5303


In [6]:
df.head()

,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1
1,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.,1
2,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...,2
3,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.,1
4,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.,1


In [7]:
df['RandomFifthSentenceQuiz1'][0],

('He is happy now.',)

In [8]:
voc.get_sentence(preprocess(df['RandomFifthSentenceQuiz1'][0]))  

[11, 12, 13, 14]

In [9]:
import sys
class NoContextDataset(Dataset):
    def __init__(self,file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv',vocab=None,df=None):

        super().__init__()
        
        created_df, created_vocab = load_data(file)
        if df is None:
            df = created_df
        if vocab:
            self.vocab = vocab
        else:
            self.vocab = created_vocab
        #self.df = df
      
        
        self.dir_st = 'data/skip-thoughts'
        self.biskip = BiSkip(self.dir_st, self.vocab.convert_to_list())
        
        self.uniskip = UniSkip(self.dir_st, self.vocab.convert_to_list())
        
        
        self.data = self.make_data(df)
        
        
    def __getitem__(self, idx):
        """
        Args:
            idx
        Returns: skip thought embedding of ending and 0/1 if it is the right ending 

        """
        return self.data[idx]

    def __len__(self):
        """
        Returns len of the dataset
        """
        return len(self.data)
       
    def make_data(self, df):
        data = []
        total = df.index
        print('skip thought encoding dataset')
        for i in total:
            #print(row['RandomFifthSentenceQuiz1'],row['RandomFifthSentenceQuiz2'])
            progress(i,len(total))
            endings =  self.gen_embbeding(df.at[i,'RandomFifthSentenceQuiz1'], df.at[i,'RandomFifthSentenceQuiz2'])
            if df.at[i,'AnswerRightEnding'] == 1:
                data.append((endings[0],1))
                data.append((endings[1],0))
            else:
                data.append((endings[0],0))
                data.append((endings[1],1))
        return data
    

    def zero_pad(self,l,n):
        l = (l + n * [0])[:n]
        return l
    
    def pad_input(self,a,b):
        ed = sorted([a,b],key=len)
        longer = ed[1]
        shorter = ed[0]
        padded = self.zero_pad( shorter,len(longer))
        if shorter == a:
            return padded,b
        else: return a,padded
        
    def gen_embbeding(self,sent1,sent2):
        sent1 = preprocess(sent1)
        sent2 = preprocess(sent2)
        #remove random n token that is in one sentence
        if 'n' in sent2:
            sent2.remove('n')
        encoded_end1 = self.vocab.get_sentence(sent1)
        encoded_end2 = self.vocab.get_sentence(sent2)
        a,b = self.pad_input(encoded_end1,encoded_end2)
        
        batch = torch.LongTensor([a,b]) 
        top_half = self.uniskip(batch)
        bottom_half = self.biskip(batch)
        combine_skip = torch.cat([top_half,bottom_half],dim=1) 
        return combine_skip    
    
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [71]:
import sys
from collections import OrderedDict 
class LastSentenceDataset(Dataset):
    '''currently does not work this class is a work in progress'''
    def __init__(self,file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv',vocab=None,df=None):

        super().__init__()
        
        created_df, created_vocab = load_data(file)
        if df is None:
            df = created_df
        if vocab:
            self.vocab = vocab
        else:
            self.vocab = created_vocab
        #self.df = df
      
        
        self.dir_st = 'data/skip-thoughts'
        self.biskip = BiSkip(self.dir_st, self.vocab.convert_to_list())
        
        self.uniskip = UniSkip(self.dir_st, self.vocab.convert_to_list())
        
        
        self.data = self.make_data(df)
        
        
    def __getitem__(self, idx):
        """
        Args:
            idx
        Returns: skip thought embedding of ending and 0/1 if it is the right ending 

        """
        return self.data[idx]

    def __len__(self):
        """
        Returns len of the dataset
        """
        return len(self.data)
       
    def make_data(self, df):
        data = []
        total = df.index
        print('skip thought encoding dataset')
        for i in total:
            #print(row['RandomFifthSentenceQuiz1'],row['RandomFifthSentenceQuiz2'])
            progress(i,len(total))
            endings =  self.gen_embbeding(df.at[i,'RandomFifthSentenceQuiz1'], 
                                          df.at[i,'RandomFifthSentenceQuiz2'],
                                          df.at[i,'InputSentence4'])
            if df.at[i,'AnswerRightEnding'] == 1:
                data.append((endings[0],1))
                data.append((endings[1],0))
            else:
                data.append((endings[0],0))
                data.append((endings[1],1))
        return data
    

    def zero_pad(self,l,n):
        l = (l + n * [0])[:n]
        return l
    
    def pad_input(self,d):
        d = OrderedDict(sorted(d.items(), key=lambda s: len(s[1])))
        for k,v in d.items():
            d[k]= self.zero_pad(v,len(list(d.items())[-1][1]))
        return d
        
    def gen_embbeding(self,sent1,sent2,last_sent):
        d = dict()
        sent1 = preprocess(sent1)
        sent2 = preprocess(sent2)
        ls = preprocess(last_sent)
        #remove random n token that is in one sentence
        if 'n' in sent2:
            sent2.remove('n')
        d['sent1'] = self.vocab.get_sentence(sent1)
        d['sent2'] = self.vocab.get_sentence(sent2)
        d['ls'] = self.vocab.get_sentence(ls)
        d = self.pad_input(d)
        
        batch = torch.LongTensor([d['sent1'],d['sent2'], d['ls']]) 
        top_half = self.uniskip(batch)
        bottom_half = self.biskip(batch)
        combine_skip = torch.cat([top_half,bottom_half],dim=1)
        end1 = combine_skip[0]
        end2 = combine_skip[1]
        ls = combine_skip[2]
        
        #print(end1[:20],end2[:20])
        end1.add_(ls)
        end2.add_(ls)
        #print('ls',ls[:20])
        #print('after',end1[:20])
        return end1,end2    
    
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [23]:
class NoContextModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input = torch.nn.Linear(4800,256)
        self.hidden= torch.nn.Linear(256,64)
        self.output = torch.nn.Linear(64,2)
        
    def forward(self, inputs):
        hidden = torch.nn.functional.relu(self.input(inputs))
        hidden1 = torch.nn.functional.relu(self.hidden(hidden))
        output = self.output(hidden1)
        return output

In [24]:
class LSModel(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hd1 = torch.nn.Linear(4800, 2400)
        self.hd2 = torch.nn.Linear(2400, 1200)
        self.hd3 = torch.nn.Linear(1200, 600)
        self.output = torch.nn.Linear(600, 2)

    def forward(self, x):
        x = torch.nn.functional.relu(self.hd1(x))
        x = torch.nn.functional.relu(self.hd2(x))
        x = torch.nn.functional.relu(self.hd3(x))
        x = self.output(x)
        #print('output',x)
        return x

In [25]:
def score(pair,model):
    '''true if model predicts right'''
    ending1, ending2 = pair
    if ending1[1] == 1:
        target = 1
    else:
        target =  2 
        
    ending1 = ending1[0]
    ending2 = ending2[0]
    if torch.cuda.is_available():
        model = model.cuda()
        ending1 = ending1.cuda()
        ending2 = ending2.cuda()
    res1 = model(ending1)
    res2 = model(ending2)
    softm = torch.nn.Softmax(dim=0)
    prob_end1_right = softm(res1)[1].item() 
    prob_end2_right = softm(res2)[1].item()
    
    if prob_end1_right > prob_end2_right:
        pred = 1
    else:
        pred = 2
    
    if pred == target:
        return True
    else: 
        #print(prob_end1_right,prob_end2_right,pred,target)
        return False
   

In [26]:
def compute_accuracy(data_set):
    num_right = 0
    for i in range(0,len(data_set),2):
        if score((data_set[i],data_set[i+1]),model):
            num_right+=1
    return num_right / (len(data_set)/2)
    

In [27]:
def save(name):
    torch.save(model.state_dict(), 'saved_models/{}.save'.format(name))

In [28]:
import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('seaborn-paper')
batch_size = 128
lr = 0.01
num_epochs = 10
report_every = 1


In [29]:
train_data_set = NoContextDataset(df=train,vocab=voc)
data_loader = torch.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True,num_workers = 0)

vocab len is 5303


/usr/cs/undergrad/2019/gnosborn/nlp/env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


skip thought encoding dataset


KeyboardInterrupt: 

In [18]:
val_data_set = NoContextDataset(file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv',df=val)

vocab len is 5303
skip thought encoding dataset


In [20]:
model = NoContextModel()
if torch.cuda.is_available():
    model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [21]:
tick = time.time()
name='NC_Adam_bach128'
epoch_losses = []
best_score= 0.5
for epoch_num in range(1, num_epochs + 1):
    batch_losses = []
    for i, batch in enumerate(data_loader):
        ### YOUR CODE BELOW ###
        # Zero the gradients
        optimizer.zero_grad()
        # Extract the inputs and the targets
        inputs, targets = batch
        # Transfer the inputs and the targets to GPUs, if available
        if torch.cuda.is_available():
            inputs = torch.FloatTensor(inputs.float()).cuda()
            targets = torch.LongTensor(targets).cuda()
        # Run the model
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs,targets)
        
        
        # Backpropagate the error
        loss.backward(retain_graph=True)
        # Update the parameters
        optimizer.step()
        
        # Append the loss
        batch_losses.append(float (loss))
        ### YOUR CODE ABOVE ###
        epoch_loss = np.mean(np.array(batch_losses))
        epoch_losses.append(epoch_loss)
        
        acc = compute_accuracy(val_data_set)
        if acc > best_score:
            best_score = acc
            print('new best saving model')
            save('{}{}_best_acc_{:.2f}'.format(name,epoch_num,acc))

        if epoch_num % report_every == 0:
            tock = time.time()
            print("Epoch {}. Loss {:.4f}. accuracy {:.4f}. Elapsed {:.0f} seconds".format(epoch_num, epoch_loss,acc, tock-tick))
    print('saving model')
    save(name+ str(epoch_num))

print("Total time elapsed: {:.0f} minutes".format((tock-tick)/60))

Epoch 1. Loss 0.6949. accuracy 0.4734. Elapsed 47 seconds
Epoch 1. Loss 0.9728. accuracy 0.4787. Elapsed 96 seconds
new best saving model
Epoch 1. Loss 1.1824. accuracy 0.5106. Elapsed 144 seconds
new best saving model
Epoch 1. Loss 1.1505. accuracy 0.5585. Elapsed 193 seconds
Epoch 1. Loss 1.0628. accuracy 0.5053. Elapsed 241 seconds
Epoch 1. Loss 1.0087. accuracy 0.5000. Elapsed 289 seconds
Epoch 1. Loss 0.9660. accuracy 0.5160. Elapsed 336 seconds
Epoch 1. Loss 0.9345. accuracy 0.5426. Elapsed 385 seconds
Epoch 1. Loss 0.9073. accuracy 0.5160. Elapsed 433 seconds
Epoch 1. Loss 0.8858. accuracy 0.5053. Elapsed 478 seconds
Epoch 1. Loss 0.8676. accuracy 0.5585. Elapsed 524 seconds
Epoch 1. Loss 0.8531. accuracy 0.5372. Elapsed 572 seconds
Epoch 1. Loss 0.8416. accuracy 0.5479. Elapsed 619 seconds
Epoch 1. Loss 0.8302. accuracy 0.5106. Elapsed 668 seconds
Epoch 1. Loss 0.8210. accuracy 0.5160. Elapsed 715 seconds


KeyboardInterrupt: 

In [20]:

train_data_set = None
test_data_set = NoContextDataset(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=voc)

vocab len is 5226
skip thought encoding dataset


In [21]:
#test_data_set= None

In [19]:
def load(name):
    l_model = NoContextModel()
    l_model.load_state_dict(torch.load('saved_models/{}.save'.format(name)))
    return l_model
#model = load('NC_adam_4_10_epoch1_best_acc_0.6277')
#print('model',model)

model NoContextModel(
  (input): Linear(in_features=4800, out_features=256, bias=True)
  (hidden): Linear(in_features=256, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=2, bias=True)
)


In [82]:

compute_accuracy(test_data_set)

0.6285408872260823

vocab len is 5303
skip thought encoding dataset


3742